SAI RAINA
USC ID = 4650068238
Github Username = saitiger

# 1(A)

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import hamming_loss, silhouette_score, accuracy_score
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings("ignore")


In [3]:
df = pd.read_csv(r'../Data/Frogs_MFCCs.csv')

In [4]:
df.drop("RecordID",axis=1,inplace=True)
df.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.024017,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,0.012022,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,0.083536,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.050224,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,0.062837,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre


In [ ]:
# df.dtypes

In [ ]:
# le = LabelEncoder()
# df['Family'] = le.fit_transform(df['Family'])
# df['Genus'] = le.fit_transform(df['Genus'])
# df['Species'] = le.fit_transform(df['Species'])

In [5]:
train, test = train_test_split(df, train_size = 0.70, random_state = 5)

In [6]:
x_train = train.drop(columns=['Family','Genus','Species'])
y_train = train[['Family','Genus','Species']]
x_test = test.drop(columns=['Family','Genus','Species'])
y_test = test[['Family','Genus','Species']]

# 1(B) (i)
**Exact Match** : When working with multi label multi class problem we have predictions for all the labels. Exact match is computed as the subset of predictions which classifies correctly on all labels meaning partially correct predictions won't be considered as a successful result. This is a harsh metric as it only rewards when we correctly predict on all the labels.

**Hamming loss** : Hamming-Loss overcomes the drawback of exact match as it takes it calculated on the basis of individual 
predictions of the labels meaning even one correct prediction out of all the labels is considered in the positive case.

Hamming loss is computed as the fraction of labels that are incorrectly predicted divided by the total number of labels.

# 1(B) (ii)

In [7]:
c_ls = np.logspace(-3, 4, 10)
gamma_ls = np.linspace(0.1, 2, 10)

params = {'gamma':gamma_ls, 'C':c_ls}
clf = SVC(kernel='rbf',decision_function_shape='ovr')
svm_gs = GridSearchCV(clf, param_grid = params, cv = 10,n_jobs = -1)

In [8]:
model_1 = svm_gs.fit(x_train, y_train['Family'])

print("Label : Family")
svm_res = pd.DataFrame(model_1.cv_results_)
print(svm_res[['params','mean_test_score']])
print("Best Parameters : ",model_1.best_params_)

pred_family = model_1.predict(x_test)
exactmatchscore_family = accuracy_score(pred_family, y_test['Family'])
exactmatchloss_family = 1-exactmatchscore_family
print('Exact Match Loss : ', exactmatchloss_family)

hammingloss_family = hamming_loss(pred_family, y_test['Family'])
print('Hamming Loss : ', hammingloss_family)

Label : Family
                                         params  mean_test_score
0                    {'C': 0.001, 'gamma': 0.1}         0.610405
1     {'C': 0.001, 'gamma': 0.3111111111111111}         0.610405
2     {'C': 0.001, 'gamma': 0.5222222222222223}         0.610405
3     {'C': 0.001, 'gamma': 0.7333333333333333}         0.610405
4     {'C': 0.001, 'gamma': 0.9444444444444444}         0.610405
..                                          ...              ...
95  {'C': 10000.0, 'gamma': 1.1555555555555557}         0.992456
96  {'C': 10000.0, 'gamma': 1.3666666666666667}         0.993052
97   {'C': 10000.0, 'gamma': 1.577777777777778}         0.993846
98   {'C': 10000.0, 'gamma': 1.788888888888889}         0.994243
99                 {'C': 10000.0, 'gamma': 2.0}         0.994044

[100 rows x 2 columns]
Best Parameters :  {'C': 46.41588833612782, 'gamma': 1.788888888888889}
Exact Match Loss :  0.0069476609541454515
Hamming Loss :  0.006947660954145438


In [9]:
model_2 = svm_gs.fit(x_train, y_train['Genus'])

print("Label : Genus")
svm_res = pd.DataFrame(model_2.cv_results_)
print(svm_res[['params','mean_test_score']])
print("Best Parameters : ",model_2.best_params_)

pred_genus = model_2.predict(x_test)
exactmatchscore_genus = accuracy_score(pred_genus, y_test['Genus'])
exactmatchloss_genus = 1-exactmatchscore_genus
print('Exact Match Loss : ', exactmatchloss_genus)

hammingloss_genus = hamming_loss(pred_genus, y_test['Genus'])
print('Hamming Loss : ', hammingloss_genus)

Label : Genus
                                         params  mean_test_score
0                    {'C': 0.001, 'gamma': 0.1}         0.574861
1     {'C': 0.001, 'gamma': 0.3111111111111111}         0.574861
2     {'C': 0.001, 'gamma': 0.5222222222222223}         0.574861
3     {'C': 0.001, 'gamma': 0.7333333333333333}         0.574861
4     {'C': 0.001, 'gamma': 0.9444444444444444}         0.574861
..                                          ...              ...
95  {'C': 10000.0, 'gamma': 1.1555555555555557}         0.989874
96  {'C': 10000.0, 'gamma': 1.3666666666666667}         0.990271
97   {'C': 10000.0, 'gamma': 1.577777777777778}         0.990469
98   {'C': 10000.0, 'gamma': 1.788888888888889}         0.990867
99                 {'C': 10000.0, 'gamma': 2.0}         0.991264

[100 rows x 2 columns]
Best Parameters :  {'C': 46.41588833612782, 'gamma': 2.0}
Exact Match Loss :  0.011579434923575715
Hamming Loss :  0.01157943492357573


In [10]:
model_3 = svm_gs.fit(x_train, y_train['Species'])

print("Label : Species")
svm_res = pd.DataFrame(model_3.cv_results_)
print(svm_res[['params','mean_test_score']])
print("Best Parameters : ",model_3.best_params_)

pred_species = model_3.predict(x_test)
exactmatchscore_species = accuracy_score(pred_species, y_test['Species'])
exactmatchloss_species = 1-exactmatchscore_species
print('Exact Match Loss : ', exactmatchloss_species)

hammingloss_species = hamming_loss(pred_species, y_test['Species'])
print('Hamming Loss : ', hammingloss_species)

Label : Species
                                         params  mean_test_score
0                    {'C': 0.001, 'gamma': 0.1}         0.478753
1     {'C': 0.001, 'gamma': 0.3111111111111111}         0.478753
2     {'C': 0.001, 'gamma': 0.5222222222222223}         0.478753
3     {'C': 0.001, 'gamma': 0.7333333333333333}         0.478753
4     {'C': 0.001, 'gamma': 0.9444444444444444}         0.478753
..                                          ...              ...
95  {'C': 10000.0, 'gamma': 1.1555555555555557}         0.989079
96  {'C': 10000.0, 'gamma': 1.3666666666666667}         0.989477
97   {'C': 10000.0, 'gamma': 1.577777777777778}         0.989477
98   {'C': 10000.0, 'gamma': 1.788888888888889}         0.990073
99                 {'C': 10000.0, 'gamma': 2.0}         0.990271

[100 rows x 2 columns]
Best Parameters :  {'C': 46.41588833612782, 'gamma': 2.0}
Exact Match Loss :  0.01204261232051873
Hamming Loss :  0.012042612320518759


# 1(B) (iii)

In [11]:
scaler = StandardScaler()
scaler = scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [12]:
c_ls = np.logspace(-3, 4, 20)

params = {'C':c_ls}
clf2 = LinearSVC(penalty = 'l2',multi_class = 'ovr',dual = False,loss = 'squared_hinge' )
linsvc_gs = GridSearchCV(clf2, param_grid=params, cv = 10,n_jobs = -1)

In [13]:
model_4 = linsvc_gs.fit(x_train_scaled, y_train['Family'])

print("L1 Penalized SVM")
print("Label : Family")
linsvc_res = pd.DataFrame(model_4.cv_results_)
print(linsvc_res[['params','mean_test_score']])
print("Best Parameters : ",model_4.best_params_)

pred_family_scaled = model_4.predict(x_test_scaled)
exactmatchscore_family_scaled = accuracy_score(pred_family_scaled, y_test['Family'])
exactmatchloss_family_scaled = 1-exactmatchscore_family_scaled
print('Exact Match Loss : ', exactmatchloss_family_scaled)

hammingloss_family_scaled = hamming_loss(pred_family_scaled, y_test['Family'])
print('Hamming Loss : ', hammingloss_family_scaled)

L1 Penalized SVM
Label : Family
                         params  mean_test_score
0                  {'C': 0.001}         0.912425
1   {'C': 0.002335721469090121}         0.926126
2    {'C': 0.00545559478116852}         0.931291
3   {'C': 0.012742749857031334}         0.933475
4   {'C': 0.029763514416313176}         0.935660
5    {'C': 0.06951927961775606}         0.936058
6     {'C': 0.1623776739188721}         0.936655
7    {'C': 0.37926901907322497}         0.937252
8     {'C': 0.8858667904100823}         0.937252
9      {'C': 2.069138081114788}         0.937649
10     {'C': 4.832930238571752}         0.937649
11    {'C': 11.288378916846883}         0.937649
12    {'C': 26.366508987303554}         0.937649
13     {'C': 61.58482110660255}         0.937649
14     {'C': 143.8449888287663}         0.937649
15     {'C': 335.9818286283781}         0.937649
16     {'C': 784.7599703514607}         0.937649
17    {'C': 1832.9807108324337}         0.937649
18     {'C': 4281.332398719387}      

In [14]:
model_5 = linsvc_gs.fit(x_train_scaled, y_train['Genus'])

print("L1 Penalized SVM")
print("Label : Genus")
linsvc_res = pd.DataFrame(model_5.cv_results_)
print(linsvc_res[['params','mean_test_score']])
print("Best Parameters : ",model_5.best_params_)

pred_genus_scaled = model_5.predict(x_test_scaled)
exactmatchscore_genus_scaled = accuracy_score(pred_genus_scaled, y_test['Genus'])
exactmatchloss_genus_scaled = 1-exactmatchscore_genus_scaled
print("Exact Match Loss : ", exactmatchloss_genus_scaled)

hammingloss_genus_scaled = hamming_loss(pred_genus_scaled, y_test['Genus'])
print("Hamming Loss : ", hammingloss_genus_scaled)

L1 Penalized SVM
Label : Genus
                         params  mean_test_score
0                  {'C': 0.001}         0.907269
1   {'C': 0.002335721469090121}         0.920974
2    {'C': 0.00545559478116852}         0.930106
3   {'C': 0.012742749857031334}         0.935865
4   {'C': 0.029763514416313176}         0.939836
5    {'C': 0.06951927961775606}         0.945396
6     {'C': 0.1623776739188721}         0.948374
7    {'C': 0.37926901907322497}         0.951155
8     {'C': 0.8858667904100823}         0.951949
9      {'C': 2.069138081114788}         0.952943
10     {'C': 4.832930238571752}         0.953539
11    {'C': 11.288378916846883}         0.954135
12    {'C': 26.366508987303554}         0.953936
13     {'C': 61.58482110660255}         0.954134
14     {'C': 143.8449888287663}         0.953339
15     {'C': 335.9818286283781}         0.952943
16     {'C': 784.7599703514607}         0.952943
17    {'C': 1832.9807108324337}         0.952943
18     {'C': 4281.332398719387}       

In [15]:
model_6 = linsvc_gs.fit(x_train_scaled, y_train['Species'])

print("L1 Penalized SVM")
print("Label : Species")
linsvc_res = pd.DataFrame(model_6.cv_results_)
print(linsvc_res[['params','mean_test_score']])
print("Best Parameters : ",model_6.best_params_)

pred_species_scaled = model_6.predict(x_test_scaled)
exactmatchscore_species_scaled = accuracy_score(pred_species_scaled, y_test['Species'])
exactmatchloss_species_scaled = 1-exactmatchscore_species_scaled
print("Exact Match Loss : ", exactmatchloss_species_scaled)

hammingloss_species_scaled = hamming_loss(pred_species_scaled, y_test['Species'])
print("Hamming Loss : ", hammingloss_species_scaled)

L1 Penalized SVM
Label : Species
                         params  mean_test_score
0                  {'C': 0.001}         0.909254
1   {'C': 0.002335721469090121}         0.922957
2    {'C': 0.00545559478116852}         0.931098
3   {'C': 0.012742749857031334}         0.937848
4   {'C': 0.029763514416313176}         0.944402
5    {'C': 0.06951927961775606}         0.952741
6     {'C': 0.1623776739188721}         0.958101
7    {'C': 0.37926901907322497}         0.959889
8     {'C': 0.8858667904100823}         0.960088
9      {'C': 2.069138081114788}         0.960684
10     {'C': 4.832930238571752}         0.960684
11    {'C': 11.288378916846883}         0.960485
12    {'C': 26.366508987303554}         0.959691
13     {'C': 61.58482110660255}         0.959691
14     {'C': 143.8449888287663}         0.959492
15     {'C': 335.9818286283781}         0.958897
16     {'C': 784.7599703514607}         0.958500
17    {'C': 1832.9807108324337}         0.958500
18     {'C': 4281.332398719387}     

# 1 (B) (iv)

In [16]:
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state = 5)],
                                ['classifier', LinearSVC(penalty = 'l2',multi_class = 'ovr',dual = False,loss = 'squared_hinge' )]])

param_grid = {'classifier__C':np.logspace(-3, 4, 10)}

grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                            n_jobs = -1)

model_7 = grid_search.fit(x_train, y_train['Family'])
cv_score = grid_search.best_score_
print("Label : Family")
print(f'Cross-validation score: {cv_score}')
print("Best Parameter : ",model_7.best_params_)

pred_family_smote = model_7.predict(x_test)
exactmatchscore_family_smote = accuracy_score(pred_family_smote, y_test['Family'])
exactmatchloss_family_smote = round(1-exactmatchscore_family_smote, 3)

hammingloss_family_smote = hamming_loss(pred_family_smote, y_test['Family'])

print("Exact Match Loss : ", exactmatchloss_family_smote)
print("Hamming Loss : ", hammingloss_family_smote)

Label : Family
Cross-validation score: 0.9197768398984885
Best Parameter :  {'classifier__C': 278.2559402207126}
Exact Match Loss :  0.084
Hamming Loss :  0.08429828624363131


In [18]:
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state = 5)],
                                ['classifier', LinearSVC(penalty = 'l2',multi_class = 'ovr',dual = False,loss = 'squared_hinge' )]])

param_grid = {'classifier__C':np.logspace(-3, 4, 10)}

grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                            n_jobs = -1)

model_8 = grid_search.fit(x_train, y_train['Genus'])
cv_score = grid_search.best_score_
print("Label : Genus")
print("Cross-validation score: ",cv_score)
print("Best Parameter : ",model_8.best_params_)

pred_genus_smote = model_8.predict(x_test)
exactmatchscore_genus_smote = accuracy_score(pred_genus_smote, y_test['Genus'])
exactmatchloss_genus_smote = round(1-exactmatchscore_genus_smote, 3)

hammingloss_genus_smote = hamming_loss(pred_genus_smote, y_test['Genus'])

print("Exact Match Loss : ", exactmatchloss_genus_smote)
print("Hamming Loss : ", hammingloss_genus_smote)

Label : Genus
Cross-validation score:  0.9164010655569742
Best Parameter :  {'classifier__C': 46.41588833612782}
Exact Match Loss :  0.082
Hamming Loss :  0.08244557665585919


In [19]:
pipeline = imbpipeline(steps = [['smote', SMOTE(random_state = 5)],
                                ['classifier', LinearSVC(penalty = 'l2',multi_class = 'ovr',dual = False,loss = 'squared_hinge' )]])

param_grid = {'classifier__C':np.logspace(-3, 4, 10)}

grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                            n_jobs = -1)

model_9 = grid_search.fit(x_train, y_train['Species'])
cv_score = grid_search.best_score_
print("Label : Species")
print("Cross-validation score: ",{cv_score})
print("Best Parameter : ",model_9.best_params_)

pred_species_smote = model_9.predict(x_test)
exactmatchscore_species_smote = accuracy_score(pred_species_smote, y_test['Species'])
exactmatchloss_species_smote = round(1-exactmatchscore_species_smote, 3)

hammingloss_species_smote = hamming_loss(pred_species_smote, y_test['Species'])

print("Exact Match Loss : ", exactmatchloss_species_smote)
print("Hamming Loss : ", hammingloss_species_smote)

Label : Species
Cross-validation score:  {0.9567105657224821}
Best Parameter :  {'classifier__C': 46.41588833612782}
Exact Match Loss :  0.045
Hamming Loss :  0.04539138490041686


In [20]:
pred_correct = 0
for i in range(0, len(y_test)):
    if (y_test['Family'].ravel()[i]==pred_family[i] and 
        y_test['Genus'].ravel()[i]==pred_genus[i] and 
        y_test['Species'].ravel()[i]==pred_species[i]):
         pred_correct+= 1
exactscore_gaussian = float(pred_correct)/len(y_test)
exactloss_gaussian = 1-exactscore_gaussian

pred_correct_scaled = 0
for i in range(0, len(y_test)):
    if (y_test['Family'].ravel()[i]==pred_family_scaled[i] and 
        y_test['Genus'].ravel()[i]==pred_genus_scaled[i] and 
        y_test['Species'].ravel()[i]==pred_species_scaled[i]):
         pred_correct_scaled+= 1
exactscore_regularized = float(pred_correct_scaled)/len(y_test)
exactloss_regularized = 1-exactscore_regularized

pred_correct_smote = 0
for i in range(0, len(y_test)):
    if (y_test['Family'].ravel()[i]==pred_family_smote[i] and 
        y_test['Genus'].ravel()[i]==pred_genus_smote[i] and 
        y_test['Species'].ravel()[i]==pred_species_smote[i]):
         pred_correct_smote+= 1
exactscore_smote = float(pred_correct_smote)/len(y_test)
exactloss_smote = 1-exactscore_smote

In [21]:
hammingloss_avg_svm = (hammingloss_family+hammingloss_genus+hammingloss_species)/3
hammingloss_avg_svm_regularized = (hammingloss_family_scaled +hammingloss_genus_scaled +hammingloss_species_scaled)/3
hammingloss_avg_svm_smote = (hammingloss_family_smote +hammingloss_genus_smote +hammingloss_species_smote)/3


print("Exact Match Loss --> ")
print("Average Exact Match Loss Gaussian Kernel : ", exactloss_gaussian)
print("Average Exact Match Loss Regularized : ", exactloss_regularized)
print("Average Exact Match Loss Smote : ", exactloss_smote)

print("\n")
print("Hamming Loss --> ")

print("Average Hamming Loss Gaussian Kernel : ", hammingloss_avg_svm)
print("Average Hamming Loss Regularized : ", hammingloss_avg_svm_regularized)
print("Average Hamming Loss Smote : ", hammingloss_avg_svm_smote)

Exact Match Loss --> 
Average Exact Match Loss Gaussian Kernel :  0.015284854099119949
Average Exact Match Loss Regularized :  0.08985641500694763
Average Exact Match Loss Smote :  0.13015284854099118


Hamming Loss --> 
Average Hamming Loss Gaussian Kernel :  0.010189902732746642
Average Hamming Loss Regularized :  0.05419175544233442
Average Hamming Loss Smote :  0.07071174926663579


Taking the average of all exact match score for all the labels it is evident that **gaussian kernel svm** has the lowest exact match loss meaning that it does the best on partial predictions. It also has the lowest hamming loss meaning it also is performing well on all three labels combined.

# 2 (A),(B),(C)

In [22]:
df2 = pd.read_csv(r'../Data/Frogs_MFCCs.csv')

In [23]:
x_df = df2.drop(['Family','Genus','Species','RecordID'], axis=1)
y_df = df2[["Family","Genus","Species"]]

In [25]:
hamming_loss_ls = []
hamming_score_ls = []
hamming_distance_ls = []
for i in range(1,51):
    silh_score = {}
    print("Iteration ",i)
    for j in range(2,51):
        kmn = KMeans(init = "random",n_clusters = j)
        pred_labels = kmn.fit_predict(x_df)
        silhouettescore = silhouette_score(x_df,pred_labels)
        silh_score[j] = silhouettescore
        best_k = max(silh_score, key= silh_score.get) # Optimal number of clusters based on silhouette score
    print(f"Optimal Number of Clusters : {best_k}\n")
    kmn_opt = KMeans(n_clusters = best_k) # Kmeans on the optimal number of clusters 
    pred_opt_labels =  kmn_opt.fit_predict(x_df)
    label_name = pd.DataFrame(pred_opt_labels,columns = ["Label"])
    df3 = pd.concat([x_df,y_df,label_name],axis=1) # Finding the majority triplets based on true values 
    maj_triplet = {}
    for a in range(0,best_k):
        df4 = df3[df3['Label']==a]
        maj_triplet[a]=[df4['Family'].value_counts().index[0],df4['Genus'].value_counts().index[0],df4['Species'].value_counts().index[0]]
        df3['family_pred'] = 'none'
        df3['genus_pred'] = 'none'
        df3['species_pred'] = 'none'
        df3['family_pred'] = np.where(df3['Label']==a,maj_triplet[a][0],df3['family_pred'])
        df3['genus_pred'] = np.where(df3['Label']==a,maj_triplet[a][1],df3['genus_pred'])
        df3['species_pred'] = np.where(df3['Label']==a,maj_triplet[a][2],df3['species_pred'])
        hamming_loss_family = hamming_loss(df3['Family'],df3['family_pred'])
        hamming_loss_genus = hamming_loss(df3['Genus'],df3['genus_pred'])
        hamming_loss_species = hamming_loss(df3['Species'],df3['species_pred'])
        hamming_loss_iter = (hamming_loss_family+hamming_loss_genus+hamming_loss_species)/3
        hamming_score_iter = (1-hamming_loss_iter)
        hamming_distance_iter = (hamming_loss_iter * 7195)
    hamming_loss_ls.append(hamming_loss_iter)
    hamming_score_ls.append(hamming_score_iter)
    hamming_distance_ls.append(hamming_distance_iter)
    print(f"The majoritry classes for Family,Genus and Species are : {maj_triplet}\n")
    print("Hamming Loss for Label Family is : ", hamming_loss_family)
    print("Hamming Loss for Label Genus is : ",hamming_loss_genus)
    print("Hamming Loss for Label Species is : ",hamming_loss_species)
    print("\n")

Iteration  1
Optimal Number of Clusters : 4

The majoritry classes for Family,Genus and Species are : {0: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 1: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae'], 2: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens'], 3: ['Dendrobatidae', 'Ameerega', 'Ameeregatrivittata']}

Hamming Loss for Label Family is :  0.9305072967338429
Hamming Loss for Label Genus is :  0.9305072967338429
Hamming Loss for Label Species is :  0.9305072967338429


Iteration  2
Optimal Number of Clusters : 4

The majoritry classes for Family,Genus and Species are : {0: ['Dendrobatidae', 'Ameerega', 'Ameeregatrivittata'], 1: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens'], 2: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 3: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']}

Hamming Loss for Label Family is :  0.826963168867269
Hamming Loss for Label Genus is :  0.855733148019458
Hamming Loss for Label Species is :  0.8585128561501042


Iteration  3
Opt

Optimal Number of Clusters : 4

The majoritry classes for Family,Genus and Species are : {0: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 1: ['Dendrobatidae', 'Ameerega', 'Ameeregatrivittata'], 2: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens'], 3: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']}

Hamming Loss for Label Family is :  0.826963168867269
Hamming Loss for Label Genus is :  0.855733148019458
Hamming Loss for Label Species is :  0.8585128561501042


Iteration  19
Optimal Number of Clusters : 4

The majoritry classes for Family,Genus and Species are : {0: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens'], 1: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 2: ['Dendrobatidae', 'Ameerega', 'Ameeregatrivittata'], 3: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae']}

Hamming Loss for Label Family is :  0.826963168867269
Hamming Loss for Label Genus is :  0.855733148019458
Hamming Loss for Label Species is :  0.8585128561501042


Iteration  20
Optimal Number o

Hamming Loss for Label Genus is :  0.9305072967338429
Hamming Loss for Label Species is :  0.9305072967338429


Iteration  35
Optimal Number of Clusters : 4

The majoritry classes for Family,Genus and Species are : {0: ['Dendrobatidae', 'Ameerega', 'Ameeregatrivittata'], 1: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 2: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae'], 3: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens']}

Hamming Loss for Label Family is :  0.9184155663655316
Hamming Loss for Label Genus is :  0.9249478804725504
Hamming Loss for Label Species is :  0.937178596247394


Iteration  36
Optimal Number of Clusters : 4

The majoritry classes for Family,Genus and Species are : {0: ['Dendrobatidae', 'Ameerega', 'Ameeregatrivittata'], 1: ['Leptodactylidae', 'Adenomera', 'AdenomeraHylaedactylus'], 2: ['Hylidae', 'Hypsiboas', 'HypsiboasCordobae'], 3: ['Hylidae', 'Hypsiboas', 'HypsiboasCinerascens']}

Hamming Loss for Label Family is :  0.9179986101459346
Hamming Loss 

### Hamming Distance Calculation
Hamming Distance is the Total number of predictions that is not correct as compared to the true label, while hamming loss is calculated as the proportion of labels that are incorrectly predicted therefore to calculate hamming distance we multiply hamming loss by the number of labels.

In [26]:
print("Average of Hamming Loss over 50 iterations : ",np.mean(hamming_loss_ls))
print("Standard Deviation of Hamming Loss over 50 iterations : ",np.std(hamming_loss_ls))
print("\n")
print("Average of Hamming Score over 50 iterations : ",np.mean(hamming_score_ls))
print("Standard Deviation of Hamming Score over 50 iterations : ",np.std(hamming_score_ls))
print("\n")
print("Average of Hamming Distance over 50 iterations : ",np.mean(hamming_distance_ls))
print("Standard Deviation of Hamming Distance over 50 iterations : ",np.std(hamming_distance_ls))

Average of Hamming Loss over 50 iterations :  0.8842751911049339
Standard Deviation of Hamming Loss over 50 iterations :  0.08973084173793229


Average of Hamming Score over 50 iterations :  0.11572480889506602
Standard Deviation of Hamming Score over 50 iterations :  0.0897308417379323


Average of Hamming Distance over 50 iterations :  6362.3600000000015
Standard Deviation of Hamming Distance over 50 iterations :  645.6134063044229


# References :
<ol>
    <li>https://mmuratarat.github.io/2020-01-25/multilabel_classification_metrics</li>
    <li>https://en.wikipedia.org/wiki/Multi-label_classification</li>
    <li>https://www.linkedin.com/pulse/hamming-score-multi-label-classification-chandra-sharat/</li>
    <li>https://stats.stackexchange.com/questions/233275/multilabel-classification-metrics-on-scikit</li>
    <li>https://www.codingninjas.com/codestudio/library/svm-hyperparameter-tuning-using-gridsearchcv</li>
    <li>https://realpython.com/k-means-clustering-python/</li>
    <li>https://www.analyticsvidhya.com/blog/2020/02/4-types-of-distance-metrics-in-machine-learning/</li>
</ol>  

Theory question submitted as pdf and uploaded to github as a separate file.